In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

In [3]:
a = np.ones([5, 8], dtype="int")

b = a[0:1]
c = a[1:]

print(a)
print(b)
print(c)

[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]
[[1 1 1 1 1 1 1 1]]
[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]


In [4]:
training_dataroot = 'lab1_basic_training.csv' # Training data file file named as 'lab1_basic_training.csv'
testing_dataroot = 'lab1_basic_testing.csv'   # Testing data file named as 'lab1_basic_testing.csv'
output_dataroot = 'lab1_basic.csv' # Output file will be named as 'lab1_basic.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 100 elements

# Read input csv to datalist
with open(training_dataroot, newline="") as csvfile:
    training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline="") as csvfile:
    testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

In [5]:
def SplitData(data, split_ratio):
    """
    Splits the given dataset into training and validation sets based on the specified split ratio.

    Parameters:
    - data (numpy.ndarray): The dataset to be split. It is expected to be a 2D array where each row represents a data point and each column represents a feature.
    - split_ratio (float): The ratio of the data to be used for training. For example, a value of 0.8 means 80% of the data will be used for training and the remaining 20% for validation.

    Returns:
    - training_data (numpy.ndarray): The portion of the dataset used for training.
    - validation_data (numpy.ndarray): The portion of the dataset used for validation.

    """
    training_data = []
    validation_data = []

    # <TODO> - Done
    data_size = len(data)
    training_data = data[: data_size * split_ratio]
    validation_data = data[data_size * split_ratio:]

    print(f"{data_size=}, training={len(training_data)=}, validation={len(validation_data)=}")

    return training_data, validation_data

In [6]:
def PreprocessData(data):
    """
    Preprocess the given dataset and return the result.

    Parameters:
    - data (numpy.ndarray): The dataset to preprocess. It is expected to be a 2D array where each row represents a data point and each column represents a feature.

    Returns:
    - preprocessedData (numpy.ndarray): Preprocessed data.
    """
    preprocessedData = data

    # <TODO>
    print(f"{preprocessedData=}")

    data_removed_nan = np.array(
        [i for i in preprocessedData if not np.isnan(i[0]) and not np.isnan(i[1])]
    )

    (x_mean, y_mean) = np.mean(data_removed_nan, axis=0).tolist()
    print(f"{(x_mean, y_mean)=}")

    x_q25, x_q75 = np.percentile(data_removed_nan[:, 0], [25, 75]).tolist()
    y_q25, y_q75 = np.percentile(data_removed_nan[:, 1], [25, 75]).tolist()
    print(f"{(x_q25, x_q75)=}, {(y_q25, y_q75)=}")

    x_iqr = x_q75 - x_q25
    y_iqr = y_q75 - y_q25
    print(f"{x_iqr=}, {y_iqr=}")

    x_ac_range = (x_q25 - 3 * x_iqr, x_q75 + 3 * x_iqr)
    y_ac_range = (y_q25 - 3 * y_iqr, y_q75 + 3 * y_iqr)
    print(f"{x_ac_range=}, {y_ac_range=}")

    for i in preprocessedData:
        # nan value
        if np.isnan(i[0]):
            i[0] = x_mean
        if np.isnan(i[1]):
            i[1] = y_mean
        
        # extreme outlier
        if not x_ac_range[0] <= i[0] <= x_ac_range[1]:
            i[0] = x_mean
        if not y_ac_range[0] <= i[1] <= y_ac_range[1]:
            i[1] = y_mean

    return preprocessedData


# print(training_datalist)
res = PreprocessData(training_datalist)
print("===")
print(res)
print(np.mean(res, 0))

preprocessedData=array([[ 55.4    ,  32.8    ],
       [ 53.6    ,  39.4    ],
       [ 78.4    ,  52.6    ],
       ...,
       [ 65.3    ,  46.     ],
       [ 62.7    , 125.68257],
       [ 59.7    ,  38.9    ]])
(x_mean, y_mean)=(68.62171854538148, 49.36334674463573)
(x_q25, x_q75)=(57.4, 75.7), (y_q25, y_q75)=(33.4, 55.6)
x_iqr=18.300000000000004, y_iqr=22.200000000000003
x_ac_range=(2.499999999999986, 130.60000000000002), y_ac_range=(-33.20000000000001, 122.20000000000002)
===
[[55.4        32.8       ]
 [53.6        39.4       ]
 [78.4        52.6       ]
 ...
 [65.3        46.        ]
 [62.7        49.36334674]
 [59.7        38.9       ]]
[66.08805168 45.14905773]


In [7]:
def Regression(dataset):
    """
    Performs regression on the given dataset and return the coefficients.

    Parameters:
    - dataset (numpy.ndarray): A 2D array where each row represents a data point.

    Returns:
    - w (numpy.ndarray): The coefficients of the regression model. For example, y = w[0] + w[1] * x + w[2] * x^2 + ...
    """

    X = dataset[:, :1]
    y = dataset[:, 1]
    print(f"{np.sum(y)=}")

    # print(f"{X=}, {y=}")

    # <TODO>: Decide on the degree of the polynomial
    degree = 2  # For example, quadratic regression

    # Add polynomial features to X
    X_poly = np.ones((X.shape[0], 1))  # Add intercept term (column of ones)
    for d in range(1, degree + 1):
        X_poly = np.hstack((X_poly, X ** d))  # Add x^d terms to feature matrix
    # print(f"{X_poly=}")

    # Initialize coefficients (weights) to zero
    num_dimensions = X_poly.shape[1]  # Number of features (including intercept and polynomial terms)
    w = np.zeros(num_dimensions)    # shihtl> 這個 w 就是我們要解的東西
    # print(f"{num_dimensions=}, {w=}")

    # <TODO>: Set hyperparameters
    num_iteration = 10000
    learning_rate = 0.000000001


    # Gradient Descent
    m = len(y)  # Number of data points
    for iteration in range(num_iteration):
        # <TODO>: Prediction using current weights and compute error
        # print(f"{X_poly.shape=}, {w.shape=}")
        y_hat = np.matmul(X_poly, w)
        d = np.subtract(y, y_hat)
        print(f"{d=}, {y_hat=}, {len(y)=}")

        # <TODO>: Compute gradient
        g = -2 * (np.matmul(np.transpose(d), X_poly) / len(y))

        # <TODO>: Update the weights
        w = w - learning_rate * g

        # <TODO>: Optionally, print the cost every 100 iterations
        # if iteration % 100 == 0:
        #     cost = ...
        #     print(f"Iteration {iteration}, Cost: {cost}")
        cost = np.sum((y - y_hat) ** 2) / len(y)
        print(f"Iteration {iteration}, Cost: {cost}\n\n")

    return w


Regression(training_datalist)

np.sum(y)=np.float64(451490.5773121955)
d=array([32.8       , 39.4       , 52.6       , ..., 46.        ,
       49.36334674, 38.9       ]), y_hat=array([0., 0., 0., ..., 0., 0., 0.]), len(y)=10000
Iteration 0, Cost: 2246.43463713442


d=array([31.45252537, 38.13865369, 49.90163368, ..., 44.12797888,
       47.63741612, 37.33526138]), y_hat=array([1.34747463, 1.26134631, 2.69836632, ..., 1.87202112, 1.72593063,
       1.56473862]), len(y)=10000
Iteration 1, Cost: 2058.3586362486967


d=array([30.16963282, 36.93776146, 47.33259557, ..., 42.34568052,
       45.99420637, 35.84551795]), y_hat=array([2.63036718, 2.46223854, 5.26740443, ..., 3.65431948, 3.36914038,
       3.05448205]), len(y)=10000
Iteration 2, Cost: 1887.8789323463582


d=array([28.94822705, 35.79442586, 44.88668718, ..., 40.64880467,
       44.42975282, 34.42717534]), y_hat=array([3.85177295, 3.60557414, 7.71331282, ..., 5.35119533, 4.93359392,
       4.47282466]), len(y)=10000
Iteration 3, Cost: 1733.349224369093


d=arra

array([6.62233797e-05, 1.89465618e-03, 9.13387538e-03])

In [12]:
a = np.array([[1, 55.4, 55.4 * 55.4], [1, 53.6, 53.6 * 53.6]])
c = np.array([1, 3, 5])
print(a @ c)
print(f"{a.shape=}, {c.shape=}")
d = -2 * c
d

[15513.  14526.6]
a.shape=(2, 3), c.shape=(3,)


array([ -2,  -6, -10])

In [15]:
a = np.array([[1, 55.4, np.nan], [1, np.nan, 53.6 * 53.6], [1, 26264, 53.6 * 53.6]])
b = [i for i in a if not np.any(np.isnan(i))]
print(b)

[array([ 1. , 55.4,  nan]), array([1.00000e+00,         nan, 2.87296e+03])]
